In [ ]:
#import statements:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import linear_sum_assignment
from sklearn.decomposition import PCA
from tabulate import tabulate
import pandas as pd
from scipy.spatial.distance import cdist

def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            if XY.shape[0] > 1:  # Ensure at least two points for interpolation
                XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def data_cleaning(points):
    """Ensure there are no NaN or infinite values."""
    points = np.nan_to_num(points, nan=0.0, posinf=0.0, neginf=0.0)
    return points

def uniform_sampling(points, num_samples):
    if len(points) < 2:
        return points  # Not enough points to interpolate
    cumulative_dist = np.cumsum(np.sqrt(np.sum(np.diff(points, axis=0)**2, axis=1)))
    cumulative_dist = np.insert(cumulative_dist, 0, 0)  # Add the starting point
    try:
        distance_fn = interp1d(cumulative_dist, points, axis=0, kind='linear')
        uniform_dist = np.linspace(0, cumulative_dist[-1], num_samples)
        return distance_fn(uniform_dist)
    except ValueError as e:
        print(f"Interpolation error: {e}")
        return points
    
uniform_points = 150

def normalisation(points, num_points=uniform_points):
    if len(points) == 0:
        return np.zeros((num_points, 2))
    points = points - np.mean(points, axis=0)
    scale = np.sqrt(np.sum(points**2, axis=1)).max()
    if scale == 0:
        return np.zeros((num_points, 2))
    points /= scale
    points = data_cleaning(points)
    return uniform_sampling(points, num_points)

def hungarian_matching(points, reference_points):
    cost_matrix = np.linalg.norm(points[:, np.newaxis] - reference_points, axis=2)
    cost_matrix = data_cleaning(cost_matrix)
    if np.any(np.isnan(cost_matrix)) or np.any(np.isinf(cost_matrix)):
        raise ValueError("Cost matrix contains NaN or infinite values.")
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return cost_matrix[row_ind, col_ind].sum()

def multi_shape(shape_files):
    all_shapes = []
    for csv_path, label in shape_files:
        paths_XYs = read_csv(csv_path)
        for path in paths_XYs:
            for XY in path:
                key_points = normalisation(XY)
                all_shapes.append((key_points, label))
    return all_shapes

def internalangles(points):
    angles = []
    for i in range(len(points)):
        p1 = points[i - 1]
        p2 = points[i]
        p3 = points[(i + 1) % len(points)]
        ba = p1 - p2
        bc = p3 - p2
        cosinea = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosinea, -1.0, 1.0))
        angles.append(np.degrees(angle))
    return angles

def classify(points, reference_shapes):
    best_label = None
    best_score = np.inf
    best_shape = None

    for reference, label in reference_shapes:
        try:
            score = hungarian_matching(points, reference)
        except ValueError as e:
            print(f"Skipping due to error: {e}")
            continue

        if score < best_score:
            best_score = score
            best_label = label
            best_shape = reference

    # Apply angle constraint only for polygons pentagon or larger
    if best_label in ['pentagon', 'hexagon', 'heptagon', 'octagon','nonagon','decagon']:
        angles = internalangles(points)
        max_angle = max(angles) if angles else 0

        if max_angle > 170 or len(angles) < 5:  # Consider it a circle if angles are too large or not enough vertices
            best_label = 'circle'
            best_shape = [shape for shape, label in reference_shapes if label == 'circle'][0]
            best_score = 0  # Reset score since we're overriding the classification


    if best_label in ['hexagon', 'heptagon', 'octagon','nonagon','decagon'] and best_score > 3:
        best_label = 'circle'
        best_shape = [shape for shape, label in reference_shapes if label == 'circle'][0]
        best_score = 0  # Reset score since we're overriding the classification
    return best_label, best_score, best_shape

def align_axis(points):
    pca = PCA(n_components=2)
    pca.fit(points)
    return pca.transform(points), pca

def align_scale(reference_shape, original_shape, shape_label):
    if shape_label in ['rectangle', 'ellipse', 'line', 'triangle']:
        aligned_ref_shape, pca_reference = align_axis(reference_shape)
        aligned_original, pca_original = align_axis(original_shape)
        xmin, ymin = aligned_original.min(axis=0)
        xmax, ymax = aligned_original.max(axis=0)
        bounding_width = xmax - xmin
        bounding_height = ymax - ymin

        ref_xmin, ref_ymin = aligned_ref_shape.min(axis=0)
        ref_xmax, ref_ymax = aligned_ref_shape.max(axis=0)
        ref_width = ref_xmax - ref_xmin
        ref_height = ref_ymax - ref_ymin
#error handling
        if ref_width == 0 or ref_height == 0:
            return original_shape, pca_original

        scale_x = bounding_width / ref_width
        scale_y = bounding_height / ref_height
        scaled_shape = aligned_ref_shape * [scale_x, scale_y]
        aligned_shape = scaled_shape + [xmin - ref_xmin * scale_x, ymin - ref_ymin * scale_y]
        aligned_shape = pca_original.inverse_transform(aligned_shape)
        return aligned_shape, pca_original
    
    else:
        xmin, ymin = original_shape.min(axis=0)
        xmax, ymax = original_shape.max(axis=0)
        bounding_width = xmax - xmin
        bounding_height = ymax - ymin
        ref_xmin, ref_ymin = reference_shape.min(axis=0)
        ref_xmax, ref_ymax = reference_shape.max(axis=0)
        ref_width = ref_xmax - ref_xmin
        ref_height = ref_ymax - ref_ymin

        # error division
        if ref_width == 0 or ref_height == 0:
            return original_shape

        scale_x = bounding_width / ref_width
        scale_y = bounding_height / ref_height
        scaled_shape = reference_shape * [scale_x, scale_y]
        aligned_shape = scaled_shape + [xmin - ref_xmin * scale_x, ymin - ref_ymin * scale_y]
        return aligned_shape, None

def add_to_contour(contour, min_points=10):
    if len(contour) >= min_points:
        return contour
    points_needed = min_points - len(contour)
    extended_contour = []
    for i in range(len(contour)):
        extended_contour.append(contour[i])
        if i < len(contour) - 1:
            next_point = contour[i + 1]
        else:
            next_point = contour[0]
        num_new_points = points_needed // len(contour) + (1 if points_needed % len(contour) > i else 0)
        for j in range(1, num_new_points + 1):
            new_point = contour[i] + (next_point - contour[i]) * (j / (num_new_points + 1))
            extended_contour.append(new_point)
        points_needed -= num_new_points
    return np.array(extended_contour)

#define reference shapes
shape_files = [
    ('/square.csv', 'square'),
    ('/star.csv', 'star'),
    ('/circle.csv', 'circle'),
    ('/ellipse.csv', 'ellipse'),
    ('/pentagon.csv', 'pentagon'),
    ('/hexagon.csv', 'hexagon'),
    ('/heptagon.csv', 'heptagon'),
    ('/octagon.csv', 'octagon'),
    ('/nonagon.csv', 'nonagon'),
    ('/decagon.csv', 'decagon'),
    ('/rectangle.csv', 'rectangle'),
    ('/triangle.csv', 'triangle'),
    ('/straight_line.csv', 'line')
]


# Define the symmetry types for known shapes
symmetry_info = {
    'square': '4 lines of reflectional symmetry, 4 rotational symmetries (90° each)',
    'nonagon': '9 lines of reflectional symmetry, 9 rotational symmetries (40° each)',
    'decagon': '10 lines of reflectional symmetry, 10 rotational symmetries (36° each)',
    'star': '5 lines of reflectional symmetry, 5 rotational symmetries (72° each)',
    'circle': 'Radial Symmetry',
    'pentagon': '5 lines of reflectional symmetry, 5 rotational symmetries (72° each)',
    'hexagon': '6 lines of reflectional symmetry, 6 rotational symmetries (60° each)',
    'heptagon': '7 lines of reflectional symmetry, 7 rotational symmetries (51.43° each)',
    'octagon': '8 lines of reflectional symmetry, 8 rotational symmetries (45° each)',
    'rectangle': '2 lines of reflectional symmetry, 2 rotational symmetries (180° each)',
    'triangle': '3 lines of reflectional symmetry, 3 rotational symmetries (120° each)',
    'line': '1 line of reflectional symmetry, 1 rotational symmetry (0°)',
    'ellipse': '2 lines of reflectional symmetry, 2 rotational symmetries (180° each)'
    
}
reference_shapes = multi_shape(shape_files)

new_path = input("Enter path to shape's CSV file: ")
new_paths_XYs = read_csv(new_path)

original_shapes = []
replaced_shapes = []
bounding_boxes = []
symmetry_table = []
irregular_shapes = []

for path in new_paths_XYs:
    for XY in path:
        if len(XY) < 10:
            XY = add_to_contour(XY)  
        normalized_points = normalisation(XY)
        best_label, best_score, best_shape = classify(normalized_points, reference_shapes)
        if best_shape is None:
            continue  

        if best_score < 10:
            symmetry_table.append([best_label, symmetry_info[best_label]])
            transformed_shape, pca = align_scale(best_shape, XY, best_label)
            bounding_boxes.append((transformed_shape, pca))
        else:
            transformed_shape = XY
            irregular_shapes.append(XY) 
        original_shapes.append(XY)
        replaced_shapes.append(transformed_shape)



def symmetry_check(irregular_shapes, extension_factor=3, symmetry_threshold=200):
    results = []
    for idx, shape in enumerate(irregular_shapes):
        if shape.shape[0] < 2:
            continue
        centroid = np.mean(shape, axis=0)
        pca = PCA(n_components=2)
        pca.fit(shape)
        components = pca.components_

        majoraxis = components[0]
        projection_major = np.dot(shape - centroid, majoraxis)
        left_major = shape[projection_major <= 0]
        right_major = shape[projection_major > 0]
        right_flipped_major = right_major - 2 * np.outer(projection_major[projection_major > 0], majoraxis)
        cost_matrix_major = cdist(left_major, right_flipped_major, metric='euclidean')
        row_ind_major, col_ind_major = linear_sum_assignment(cost_matrix_major)
        hungarian_major = cost_matrix_major[row_ind_major, col_ind_major].sum()
        symmetry_major = 1 if hungarian_major < symmetry_threshold else 0

        minoraxis = components[1]
        projection_minor = np.dot(shape - centroid, minoraxis)
        left_minor = shape[projection_minor <= 0]
        right_minor = shape[projection_minor > 0]
        right_flipped_minor = right_minor - 2 * np.outer(projection_minor[projection_minor > 0], minoraxis)
        cost_matrix_minor = cdist(left_minor, right_flipped_minor, metric='euclidean')
        row_ind_minor, col_ind_minor = linear_sum_assignment(cost_matrix_minor)
        hungarian_minor = cost_matrix_minor[row_ind_minor, col_ind_minor].sum()
        symmetry_minor = 1 if hungarian_minor < symmetry_threshold else 0
        results.append([idx + 1, symmetry_major + symmetry_minor])

    results_df = pd.DataFrame(results, columns=['Irregular Shape ID', 'Number of Symmetries (LOS)'])
    return results_df

symmetry_results_df = symmetry_check(irregular_shapes)


fig, ax = plt.subplots(figsize=(6, 4))
for XY in original_shapes:
    ax.plot(XY[:, 0], XY[:, 1], 'black', alpha=0.5)

plt.title('Original Shape')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))
for transformed_shape in replaced_shapes:
    ax.plot(transformed_shape[:, 0], transformed_shape[:, 1], 'blue', alpha=0.5)

plt.title('Regularised Shapes')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

fig, ax = plt.subplots(figsize=(6, 4))
for idx, XY in enumerate(irregular_shapes):
    ax.plot(XY[:, 0], XY[:, 1], 'red', alpha=0.5)
    centroid = np.mean(XY, axis=0)
    ax.text(centroid[0], centroid[1], f'ID {idx + 1}', color='red', fontsize=8, ha='center')

plt.title('Irregular Shapes that were not changed')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()